In [1]:
import pickle
import json
import os
from glob import glob
from tqdm import tqdm
from datasets import Audio
from collections import defaultdict

sr = 16000
audio = Audio(sampling_rate=sr)

In [2]:
pkls = sorted(glob('/home/husein/ssd2/processed-youtube/*.pkl'))
len(pkls)

2541

In [3]:
with open('mapping-youtube-speakers-70.json') as fopen:
    unique_speakers = json.load(fopen)
    
len(unique_speakers)

9304

In [4]:
speakers = defaultdict(int)

for pkl in tqdm(pkls):
    with open(pkl, 'rb') as fopen:
        data = pickle.load(fopen)
        
    filename = os.path.split(pkl)[1].replace('.pkl', '')
        
    for result in data:
        speaker = result['classification_model'][1]
        y = audio.decode_example(audio.encode_example(result['wav_data']))['array']
        speaker_name = f'{filename}-{speaker}'
        if speaker_name not in unique_speakers:
            continue
        actual_speaker = unique_speakers[speaker_name]
        
        speakers[actual_speaker] += (len(y) / sr)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2541/2541 [23:46<00:00,  1.78it/s]


In [9]:
import IPython.display as ipd
ipd.Audio(y, rate = sr)

In [12]:
with open('lengths-70.json', 'w') as fopen:
    json.dump(speakers, fopen)